[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vrra/FGAN-Build-a-thon-2022/blob/main/Notebooks2022/build_a_thon_graph_v1.ipynb)

## Build-a-thon 2022, Hoist the colors?

In [1]:
!rm -rf /tmp/FGAN-Build-a-thon-2022/
!git clone https://github.com/vrra/FGAN-Build-a-thon-2022.git /tmp/FGAN-Build-a-thon-2022


Cloning into '/tmp/FGAN-Build-a-thon-2022'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 339 (delta 63), reused 63 (delta 59), pack-reused 269
Receiving objects: 100% (339/339), 2.54 MiB | 3.40 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [2]:
from IPython.display import Image
Image(filename='/tmp/FGAN-Build-a-thon-2022/Banner/1920x1080-FG_AN-buildathon_Sep02-01.png', width=500) 


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/FGAN-Build-a-thon-2022/Banner/1920x1080-FG_AN-buildathon_Sep02-01.png'

In [ ]:
!rm -rf /tmp/FGAN-Build-a-thon-2022/

## Introduction

Created: 14 Apr 2022.   
Author: Vishnu Ram OV.   
vishnu.n@ieee.org.   
Licence: available for all purposes under the sun  
but with acknowledgement and citation to "ITU FG AN Build-a-thon 2022"


## References 
[1] Neo4j Docs, "Creating an instance", https://neo4j.com/docs/aura/auradb/getting-started/create-database/   
[2] ITU-T Focus Group Autonomous Networks, "Use cases for Autonomous Networks", https://extranet.itu.int/sites/itu-t/focusgroups/an/output/FGAN-O-013-R1.docx  
[3] Build-a-thon github, https://github.com/vrra/FGAN-Build-a-thon-2022

## Prerequisite: 
create a free account at https://console.neo4j.io  
Follow steps at https://extranet.itu.int/sites/itu-t/focusgroups/an/input/FGAN-I-253.pdf  


## General cautions
Property names have strict conventions and rules in neo4j (apparently)

### For Colab users:

If you are using colab, add your credentials here and uncomment these  
and skip to step-1 below.

In [ ]:
#uri = "your uri from console.neo4j.io"  
#user = "your username from console.neo4j.io"  
#password = "your passwd from console.neo4j.io"  

### For non-colab users:
Create a local file .env with   
neo4j_uri="your uri from console.neo4j.io"  
neo4j_user="your username from console.neo4j.io"  
neo4j_password="your passwd from console.neo4j.io"  

#### Read the neo4j URI, username, passwd


In [3]:
import os
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [4]:

#Vishnu: 1 Jun 2022: use the details from https://console.neo4j.io
#especially use your own DB instance and corresponding passwd.
#if you dont have a .env file, replace here with your credentials.

uri = os.getenv('neo4j_uri')
user = os.getenv('neo4j_user')
password = os.getenv('neo4j_password')

## step-1: install neo4j

In [5]:
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.


## step-2: imports and init

In [6]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

## step-3: define helper class

In [7]:


class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    #Vishnu: 1 Aug 2022: uses labels instead of properties
    #Vishnu: 1 Aug 2022: Invariant: only 1 actor with 1 name.
    #        (Note that rel still uses properties and not labels)
    def create_node_with_usecase_label(self, actor_name, usecase_id):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node_label, actor_name)
            if not node_already_exists:  
                #node itself doesnt exist (not to mention the use case label), Lets add.
                result = session.write_transaction(
                    self._create_and_return_node_label, actor_name, usecase_id)
                for row in result:
                    print("Created node: {n1}".format(n1=row['n1name']))
            else:
                for row in node_already_exists:
                    print("Node already exists: {name} in {usecase}".format(name=row['n1name'], 
                                                                 usecase=row['n1usecase']))
                    if not (usecase_id in row['n1usecase']):
                      print("adding label "
                                +usecase_id+ 
                                " in addition to {usecase}".format(usecase=row['n1usecase']))
                      result = session.write_transaction(
                                self._add_usecase_and_return_existing_node_label, actor_name, usecase_id)
                    else:
                      print("ignoring ...")
                    #row has only 1 entry due to invariant. So break here. and exit f()
                    break
    
    #Vishnu: 1 Aug 2022: created this f() to query nodes based on a name
    #                    and return with usecase labels
    @staticmethod
    def _find_and_return_existing_node_label(tx, actor_name):
        query = (
            "MATCH (n1) "
            "WHERE (n1.name ='" + actor_name + "') "
            "RETURN n1.name as n1name, labels(n1) as n1usecase"
        )
        result = tx.run(query)
        try:
            return [{"n1name": row["n1name"],"n1usecase": row["n1usecase"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: created this f() to create nodes with usecase label
    @staticmethod
    def _create_and_return_node_label(tx, actor_name, usecase_id):
            query = (
            "CREATE (n1 :" + usecase_id + " { name: '" + actor_name + "'}) "
            "RETURN n1"
            )
            result = tx.run(query, actor_name=actor_name)
            try:
                return [{"n1name": row["n1"]["name"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: created this f() to add use case labels to existing nodes
    #                    Note that the existing labels are left there.
    @staticmethod
    def _add_usecase_and_return_existing_node_label(tx, actor_name, usecase_id):
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1 :" + usecase_id +" "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: created this f() to return all use case actors based on labels
    #                    as against properties
    @staticmethod
    def _find_and_return_all_usecase_actors_label(tx, usecase_id):
        query = (
            "MATCH (n) "
            "WHERE '" +usecase_id+ "' in labels(n) "
            "RETURN n.name AS name"
        )
        result = tx.run(query)
        return [row["name"] for row in result]

    #Vishnu: 1 Aug 2022: created this f() to list all nodes with usecase label
    #                    instead of properties
    def find_all_usecase_actors_label(self, usecase_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_all_usecase_actors_label, usecase_id)
            i=0
            for row in result:
                i=i+1
                print("Found actor-"+ str(i) +": {row}".format(row=row))

    #Vishnu: 1 Aug 2022: Created to use label for use case.
    #CAUTION: Assumes that actors already exists with label as use case.
    #So, this has to be called only in conjunction with create_node_with_usecase
    #otherwise you may end up with actors not having use case prop
    #but rels will have use case prop. not a good idea.
    def create_rel_with_usecase_label(self, actor1_name, rel_name, actor2_name, usecase_id):
        with self.driver.session() as session:
            this_rel_already_exists = session.read_transaction(
                self._find_and_return_this_existing_rels, actor1_name, rel_name, actor2_name, usecase_id)
            if (this_rel_already_exists):
                print(rel_name + " already exists with "+ usecase_id + " between "+ actor1_name + " and "+ actor2_name)
                return
            else:
                #we are pretty sure something didnt match
                #this relation does not exist currently between the nodes under the use case, 
                #add it.
                result3 = session.write_transaction(
                    self._create_and_return_rel_label, actor1_name, rel_name, actor2_name, usecase_id)
                #if (not retult3)
                for row in result3:
                    print("Created relation: {actor1} - {rel} - {actor2} in {usecase}".
                            format(actor1=row['n1name'], 
                            rel=row['r1name'],
                            actor2=row['n2name'],
                            usecase=row['r1usecase']))

    #Vishnu: 1 Aug 2022 
    #assumption: nodes already exist
    #modified from _create_and_return_rel to use labels for n.usecase
    @staticmethod
    def _create_and_return_rel_label(tx, actor1_name, rel_name, actor2_name, usecaseid):
            query = (
            "MATCH "
            "(n1), "
            "(n2) "
            "WHERE n1.name = '" + actor1_name + "' AND n2.name = '"+actor2_name+"' "
            " AND '"+ usecaseid+"' in labels(n1) "
            " AND '"+ usecaseid+"' in labels(n2) "
            "CREATE (n1)-[r1:Relation " + "{name: '"+rel_name+"', usecase: '"+usecaseid+"'} ]->(n2) "
            "RETURN n1, r1, n2"
            )
            result = tx.run(query)
            try:
                return [{"n1name": row["n1"]["name"], 
                        "r1name": row["r1"]["name"],
                        "n2name": row["n2"]["name"],
                        "r1usecase": row["r1"]["usecase"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for n.usecases.
    #Vishnu: 1 Aug 2022: Invariant: only 1 actor with 1 name.
    #Vishnu: 22 Jun 2022: refactored, uses properties instead of labels
    #Vishnu: 1 Jun 2022: created this f() to create nodes with usecase label
    #Vishnu: 2 Jun 2022: added- check if it already exists.
    #Vishnu: 2 Jun 2022: added- check the usecase duplication, adds a usecase if it doesnt exist.
    #otherwise ignores the command.
    def create_node_with_usecase(self, actor_name, usecase_id):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node, actor_name)
            if not node_already_exists:  
                result = session.write_transaction(
                    self._create_and_return_node, actor_name, usecase_id)
                for row in result:
                    print("Created node: {n1}".format(n1=row['n1name']))
            else:
                for row in node_already_exists:
                    print("Node already exists: {name} in {usecase}".format(name=row['n1name'], 
                                                                 usecase=row['n1usecase']))
                    s=row['n1usecase'].split(',')
                    if not (usecase_id in s):
                      print("adding prop "+usecase_id+ " in addition to "+row['n1usecase'])
                      result = session.write_transaction(
                                self._add_usecase_and_return_existing_node, actor_name, usecase_id)
                    else:
                      print("ignoring ...")
                    #row has only 1 entry due to invariant. So break here. and exit f()
                    break

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    #Vishnu: 2 Jun 2022: created this f() to query nodes and return with usecase prop
    @staticmethod
    def _find_and_return_existing_node(tx, actor_name):
        query = (
            "MATCH (n1) "
            "WHERE (n1.name ='" + actor_name + "') "
            "RETURN n1.name as n1name, n1.usecase as n1usecase"
        )
        result = tx.run(query)
        try:
            return [{"n1name": row["n1name"],"n1usecase": row["n1usecase"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    @staticmethod
    def _create_and_return_node(tx, actor_name, usecase_id):
            query = (
            "CREATE (n1 "+"{ name: '"+actor_name+"', usecase: '"+usecase_id+"'}) "
            "RETURN n1"
            )
            result = tx.run(query, actor_name=actor_name)
            try:
                return [{"n1name": row["n1"]["name"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    #Vishnu: 1 Jun 2022: created this f() to add use case prop to existing nodes 
    #CAUTION: does not check for duplication, use create_node_with_prop instead.
    def add_usecase_existing_node(self, actor_name, usecase_id):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._add_usecase_and_return_existing_node, actor_name, usecase_id)
            
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added usecase to node: {n1}".format(n1=row['n1']))

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    @staticmethod
    def _add_usecase_and_return_existing_node(tx, actor_name, usecase_id):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1.usecase = n1.usecase + '" + "," + "'"+ "+ '"+usecase_id +"' "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    #CAUTION: Assumes that actors already exists with property as n.use case.
    #So, this has to be called only in conjunction with create_node_with_usecase
    #otherwise you may end up with actors not having use case prop
    #but rels will have use case prop. not a good idea.
    def create_rel_with_usecase(self, actor1_name, rel_name, actor2_name, usecase_id):
        with self.driver.session() as session:
            this_rel_already_exists = session.read_transaction(
                self._find_and_return_this_existing_rels, actor1_name, rel_name, actor2_name, usecase_id)
            if (this_rel_already_exists):
                print(rel_name + " already exists with "+ usecase_id + " between "+ actor1_name + " and "+ actor2_name)
                return
            else:
                #we are pretty sure something didnt match
                #this relation does not exist currently between the nodes under the use case, 
                #add it.
                result3 = session.write_transaction(
                    self._create_and_return_rel, actor1_name, rel_name, actor2_name, usecase_id)
                #if (not retult3)
                for row in result3:
                    print("Created relation: {actor1} - {rel} - {actor2} in {usecase}".
                            format(actor1=row['n1name'], 
                            rel=row['r1name'],
                            actor2=row['n2name'],
                            usecase=row['r1usecase']))
    
    #Vishnu: 1 Aug 2022: at this moment this f() uses labels instead of properties for n.usecases.
    #                    Note that rels still uses properties.
    #        1 Aug 2022: TBD: create a property-version (as against label) of the same thing.
    #Vishnu: 21 Jun 2022: created this f() to create relationships with usecase label
    #if this works, this is the only function that needed to be called.
    def create_actors_relationship_with_usecase(self, actor1_name, rel_name, actor2_name, usecase_id):
            #process actor1
            self.create_node_with_usecase_label(actor1_name, usecase_id)
            #process actor2
            self.create_node_with_usecase_label(actor2_name, usecase_id)
            #process relation
            self.create_rel_with_usecase_label(actor1_name, rel_name, actor2_name, usecase_id)

    #Vishnu: 23 June 2022: created
    #looks for a rel under a use case between 2 given nodes.
    #normally, we expect only 0 or 1 unique entry.
    @staticmethod                
    def _find_and_return_this_existing_rels(tx, actor1_name, rel_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.usecase CONTAINS '"+ usecase_id+ "') "
            "and   (r1.name ='" + rel_name + "') "
            "RETURN n1.name as n1name, r1.name as r1name, n2.name as n2name, r1.usecase as r1usecase"
        )
        result = tx.run(query)
        return [row["r1name"] for row in result]
        
    
    #Vishnu: 21 June 2022: created this supporting f()
    #return for all relationship between given two nodes.
    @staticmethod                
    def _find_and_return_all_existing_rels_for_uc(tx, actor1_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.usecase CONTAINS '"+ usecase_id+ "') "
            "RETURN n1.name as n1name, r1.name as r1name, n2.name as n2name, r1.usecase as r1usecase"
        )
        result = tx.run(query)
        try:
            return [{
                        "n1name": row["n1name"],
                        "n2name": row["n2name"],
                        "r1name": row["r1name"],
                        "r1usecase": row["r1usecase"],
                    } 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 21 June 2022: created this supporting f()
    #adds a name for an existing relationship between given two nodes.
    @staticmethod                
    def _add_usecase_and_return_existing_rel(tx, actor1_name, rel_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "')"
            "set    r1.usecase = r1.usecase + '" + "," + "'"+ "+ '"+usecase_id +"' "
        )
        result = tx.run(query)

    
    #Vishnu: 30 june 2022: created
    #CAUTION: if the property already exists, 
    #this overwrites the current value.
    @staticmethod
    def _write_property_and_return_existing_node(tx, actor_name, propertyName, propertyVal):
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1."+propertyName+" = '"+propertyVal +"' "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 30 june 2022: created
    #CAUTION: if the property already exists, 
    #this overwrites the current value.
    @staticmethod
    def _write_property_and_return_existing_rel(tx, actor1_name, rel_name, actor2_name, 
                                                propertyName, propertyVal):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.name ='" + rel_name + "') "
            "set   r1."+propertyName+" = '"+propertyVal +"' "
            "return r1"
        )
        result = tx.run(query)
        try:
            return [{"r1": row["r1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 22 June 2022: created
    #assumption: nodes already exist
    @staticmethod
    def _create_and_return_rel(tx, actor1_name, rel_name, actor2_name, usecaseid):
            query = (
            "MATCH "
            "(n1), "
            "(n2) "
            "WHERE n1.name = '" + actor1_name + "' AND n2.name = '"+actor2_name+"' "
            " AND n1.usecase CONTAINS '"+ usecaseid+"' "
            " AND n2.usecase CONTAINS '"+ usecaseid+"' "
            "CREATE (n1)-[r1:Relation " + "{name: '"+rel_name+"', usecase: '"+usecaseid+"'} ]->(n2) "
            "RETURN n1, r1, n2"
            )
            result = tx.run(query)
            try:
                return [{"n1name": row["n1"]["name"], 
                        "r1name": row["r1"]["name"],
                        "n2name": row["n2"]["name"],
                        "r1usecase": row["r1"]["usecase"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Jun 2022: created this f() to list all nodes with usecase label
    def find_all_usecase_actors(self, usecase_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_all_usecase_actors, usecase_id)
            i=0
            for row in result:
                i=i+1
                print("Found actor-"+ str(i) +": {row}".format(row=row))

    @staticmethod
    def _find_and_return_all_usecase_actors(tx, usecase_id):
        query = (
            "MATCH (n) "
            "WHERE (n.usecase = '"+usecase_id+"') "
            "RETURN n.name AS name"
        )
        result = tx.run(query)
        return [row["name"] for row in result]

    #Vishnu: 21 Jun 2022: created this f() to cleanup the DB
    #CAUTION: will delete everything
    def cleanup_db(self):
        with self.driver.session() as session:
            result = session.read_transaction(
                self._count_nodes_in_db)
            for row in result:
                print("Found {row} actors, deleting! ".format(row=row['count']))
            result = session.write_transaction(
                self._cleanup_db)

    #Vishnu: 22 June 2022: created this f() 
    #(over)writes a prop to a node
    def writePropToNode(self, actor_name, propertyName, propertyVal):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._write_property_and_return_existing_node, 
                    actor_name, 
                    propertyName, 
                    propertyVal)
        
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added prop "+
                        propertyName+
                        " = "+
                        propertyVal+
                        " to node: {n1}".format(n1=row['n1']))

    #Vishnu: 22 June 2022: created this f() 
    #(over)writes a prop to a rel
    def writePropToRel(self, a1, rel_name, a2, propertyName, propertyVal):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._write_property_and_return_existing_rel, 
                    a1, rel_name, a2,
                    propertyName, 
                    propertyVal)
        
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added prop "+
                        propertyName+
                        " = "+
                        propertyVal+
                        " to rel: {r1}".format(r1=row['r1']))

    @staticmethod
    def _cleanup_db(tx):
        query = (
            "MATCH (n) "
            "detach delete n"
        )
        result = tx.run(query)
        try:
            return 
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    @staticmethod
    def _count_nodes_in_db(tx):
        query = (
            "MATCH (n) "
            "return count(n) as count"
        )
        result = tx.run(query)
        try:
            return [{"count": row["count"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

## step-4: instantiate the helper class to DB. 
CAUTION: this uses the username/passwd assigned already in step-2 above.

In [8]:
app = App(uri, user, password)

## step-5 clean up DB
#Caution: lose data


In [ ]:
app.cleanup_db()

## Examples

In [ ]:
app.create_actors_relationship_with_usecase("Experimentation Controller", 
                                            "myrel", 
                                            "Sandbox",
                                            "usecase_001")

### Example-1: Create individual nodes with usecase
(and add a node property)
(and add a rel property)

In [ ]:
app.create_node_with_usecase("Experimentation Controller", "usecase_001")
app.create_node_with_usecase("Sandbox", "usecase_001")


In [ ]:
app.writePropToNode("Sandbox", "capability", "import knowledge")


In [ ]:
app.writePropToNode("Sandbox", "requirement", "integrate controllers")


### Example-1a: try to create duplicate node under different use case

In [ ]:
app.create_node_with_usecase("Sandbox", "usecase_003")
app.create_node_with_usecase("Sandbox", "usecase_004")
app.create_node_with_usecase("Sandbox", "usecase_005")
app.create_node_with_usecase("Sandbox", "usecase_006")


### Example-1b: use create_node_with_usecase_label (instead of properties)

In [ ]:
app.create_node_with_usecase_label("Sandbox", "usecase_003")
app.create_node_with_usecase_label("Sandbox", "usecase_003")
app.create_node_with_usecase_label("Sandbox", "usecase_004")
app.create_node_with_usecase_label("Sandbox", "usecase_004")
app.create_node_with_usecase_label("Sandbox", "usecase_005")



### Example-2: Create relation between 2 existing nodes

In [ ]:
app.create_rel_with_usecase("Experimentation Controller", "my_best_buddy", "Sandbox", "usecase_001")
app.create_rel_with_usecase("Experimentation Controller", "my_best_buddy", "Sandbox", "usecase_002")
app.create_rel_with_usecase("Experimentation Controller", "my_best_buddy", "Sandbox", "usecase_003")


In [ ]:
app.writePropToRel("Experimentation Controller", "my_best_buddy", "Sandbox", 
                    "my_rel_prop_name", 
                    "my-rel-prop-val")

## Cleanup DB 
CAUTION: Remember to clean up the DB before executing the next steps. (so that all examples above are not leftover in the use case data below).

In [ ]:
app.cleanup_db()

## USECASES

### FG-AN-USECASE-001

In [ ]:

app.create_actors_relationship_with_usecase("Knowledge Base", "import", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("Knowledge Base", "export", "Knowledge Base", "usecase_001")

app.create_actors_relationship_with_usecase("Knowledge Base Manager", "optimizes", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("Knowledge Base Manager", "export", "Knowledge Base", "usecase_001")

app.create_actors_relationship_with_usecase("AN Orchestrator", "refer", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("AN Orchestrator", "input", "Use case description", "usecase_001")
app.create_actors_relationship_with_usecase("AN Orchestrator", "output", "TOSCA format", "usecase_001")

app.create_actors_relationship_with_usecase("Auto controller generator", "refer", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("Auto controller generator", "output", "TOSCA format", "usecase_001")

app.create_actors_relationship_with_usecase("controller repository", "stores", "controllers", "usecase_001")
app.create_actors_relationship_with_usecase("ML model", "recommends", "controllers", "usecase_001")

app.create_actors_relationship_with_usecase("Human operator", "input", "reports", "usecase_001")
app.create_actors_relationship_with_usecase("Human operator", "monitors", "reports", "usecase_001")

In [6]:
app.find_all_usecase_actors_label("usecase_001")

Found actor-1: Knowledge Base
Found actor-2: Knowledge Base Manager
Found actor-3: AN Orchestrator
Found actor-4: Use case description
Found actor-5: TOSCA format
Found actor-6: Auto controller generator
Found actor-7: controller repository
Found actor-8: controllers
Found actor-9: ML model
Found actor-10: Human operator
Found actor-11: reports


### FG-AN-USECASE-002


In [ ]:
app.create_actors_relationship_with_usecase("Experimentation controller", "decides", 
                                        "scenarios for experimentation", "usecase_002")
app.create_actors_relationship_with_usecase("AN sandbox", "hosts", 
                                        "simulators", "usecase_002")
app.create_actors_relationship_with_usecase("AN sandbox", "manages", 
                                        "simulators", "usecase_002")
app.create_actors_relationship_with_usecase("AN sandbox", "output", 
                                        "reports", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "triggers", 
                                        "experiments", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "output", 
                                        "reports", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "executes", 
                                        "experiments", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "monitors", 
                                        "experiments", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "uses", 
                                        "AN sandbox", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "updates", 
                                        "Knowledge Base", "usecase_002")
app.create_actors_relationship_with_usecase("experiments", "has", 
                                        "scenarios for experimentation", "usecase_002")

In [7]:
app.find_all_usecase_actors_label("usecase_002")

Found actor-1: Knowledge Base
Found actor-2: reports
Found actor-3: Experimentation controller
Found actor-4: scenarios for experimentation
Found actor-5: AN sandbox
Found actor-6: simulators
Found actor-7: experiments


### FG-AN-USECASE-003


In [ ]:
app.create_actors_relationship_with_usecase("AN Orchestrator", "queries", 
                                        "AN Orchestrator", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "queries", 
                                        "Human operator", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "accept", 
                                        "feedback", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "configures", 
                                        "AN components", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "updates", 
                                        "Knowledge Base", "usecase_003")


In [8]:
app.find_all_usecase_actors_label("usecase_003")

Found actor-1: Knowledge Base
Found actor-2: AN Orchestrator
Found actor-3: Human operator
Found actor-4: feedback
Found actor-5: AN components


### FG-AN-USECASE-004

In [ ]:
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", 
                                        "controllers", "usecase_004")
app.create_actors_relationship_with_usecase("Adaptation controller", "deploy", 
                                        "controllers", "usecase_004")
app.create_actors_relationship_with_usecase("Adaptation controller", "configures", 
                                        "controllers", "usecase_004")
app.create_actors_relationship_with_usecase("Adaptation controller", "queries", 
                                        "controller repository", "usecase_004")          
app.create_actors_relationship_with_usecase("Adaptation controller", "updates", 
                                        "Knowledge Base", "usecase_004")  
app.create_actors_relationship_with_usecase("controllers", "is", 
                                        "undeployed controllers", "usecase_004")  
app.create_actors_relationship_with_usecase("controllers", "is", 
                                        "deployed controllers", "usecase_004")                                         

In [9]:
app.find_all_usecase_actors_label("usecase_004")

Found actor-1: Knowledge Base
Found actor-2: controller repository
Found actor-3: controllers
Found actor-4: Adaptation controller
Found actor-5: undeployed controllers
Found actor-6: deployed controllers


### FG-AN-USECASE-005

In [ ]:
app.create_actors_relationship_with_usecase("Service management framework", "is", 
                                        "underlay", "usecase_005")
app.create_actors_relationship_with_usecase("Adaptation controller", "discover", 
                                        "Service management framework", "usecase_005")     
app.create_actors_relationship_with_usecase("Adaptation controller", "influence", 
                                        "Service management framework", "usecase_005")  
app.create_actors_relationship_with_usecase("intent", "is", 
                                        "influence", "usecase_005")  
app.create_actors_relationship_with_usecase("policy", "is", 
                                        "influence", "usecase_005")                                         

In [10]:
app.find_all_usecase_actors_label("usecase_005")

Found actor-1: Adaptation controller
Found actor-2: Service management framework
Found actor-3: underlay
Found actor-4: intent
Found actor-5: influence
Found actor-6: policy


### FG-AN-USECASE-006 cat1

In [ ]:
app.create_actors_relationship_with_usecase("Evolution controller", "monitors", 
                                        "controllers", "usecase_006_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "discovers", 
                                        "use cases", "usecase_006_cat1")     
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", 
                                        "controllers", "usecase_006_cat1")  
app.create_actors_relationship_with_usecase("Evolution controller", "refers", 
                                        "Knowledge Base", "usecase_006_cat1")  

In [11]:
app.find_all_usecase_actors_label("usecase_006_cat1")

Found actor-1: Knowledge Base
Found actor-2: controllers
Found actor-3: Evolution controller
Found actor-4: use cases


### FG-AN-USECASE-006 cat2

In [ ]:
app.create_actors_relationship_with_usecase("Data collector", "consumes", 
                                        "data", "usecase_006_cat2")
app.create_actors_relationship_with_usecase("Management domain", "produces", 
                                        "data", "usecase_006_cat2")     
app.create_actors_relationship_with_usecase("Management domain", "has", 
                                        "RAN cell", "usecase_006_cat2")  
app.create_actors_relationship_with_usecase("Data collector", "produces", 
                                        "data", "usecase_006_cat2") 

app.create_actors_relationship_with_usecase("data", "has", 
                                        "real time", "usecase_006_cat2")     
app.create_actors_relationship_with_usecase("data", "has", 
                                        "data quality", "usecase_006_cat2")  
app.create_actors_relationship_with_usecase("data", "has", 
                                        "KPI", "usecase_006_cat2") 

app.create_actors_relationship_with_usecase("Data monitor", "consumes", 
                                        "data", "usecase_006_cat2")     
app.create_actors_relationship_with_usecase("Data monitor", "has", 
                                        "data monitoring", "usecase_006_cat2")  
app.create_actors_relationship_with_usecase("Data monitor", "produces", 
                                        "reports", "usecase_006_cat2") 
app.create_actors_relationship_with_usecase("reports", "has", 
                                        "visualizations", "usecase_006_cat2")                                         
app.create_actors_relationship_with_usecase("Data analytics", "consumes", 
                                        "data", "usecase_006_cat2")    
app.create_actors_relationship_with_usecase("Data analytics", "produces", 
                                        "inference", "usecase_006_cat2")                                           
app.create_actors_relationship_with_usecase("Data analytics", "has", 
                                        "ML model", "usecase_006_cat2") 
app.create_actors_relationship_with_usecase("inference", "has", 
                                        "periodicity", "usecase_006_cat2")      
app.create_actors_relationship_with_usecase("ML model", "has", 
                                        "model training", "usecase_006_cat2")

In [12]:
app.find_all_usecase_actors_label("usecase_006_cat2")

Found actor-1: ML model
Found actor-2: reports
Found actor-3: Data collector
Found actor-4: data
Found actor-5: Management domain
Found actor-6: RAN cell
Found actor-7: real time
Found actor-8: data quality
Found actor-9: KPI
Found actor-10: Data monitor
Found actor-11: data monitoring
Found actor-12: visualizations
Found actor-13: Data analytics
Found actor-14: inference
Found actor-15: periodicity
Found actor-16: model training


### FG-AN-USECASE-007 cat1

In [ ]:
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers deployed", 
                                        "CL automation framework", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "deploys", 
                                        "controllers", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "automatic data acquisition", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "ML model", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "policy", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "action implementation", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("policy", "need", 
                                        "optimization", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "configures", 
                                        "controllers", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", 
                                        "controllers", "usecase_007_cat1")                                       
app.create_actors_relationship_with_usecase("Adaptation controller", "adapts", 
                                        "controllers", "usecase_007_cat1") 
app.create_actors_relationship_with_usecase("Adaptation controller", "interfaces with", 
                                        "underlay orchestrator", "usecase_007_cat1")   

In [13]:
app.find_all_usecase_actors_label("usecase_007_cat1")

Found actor-1: controllers
Found actor-2: ML model
Found actor-3: Adaptation controller
Found actor-4: policy
Found actor-5: CL automation framework
Found actor-6: automatic data acquisition
Found actor-7: action implementation
Found actor-8: optimization
Found actor-9: underlay orchestrator


### FG-AN-USECASE-007 cat2

In [ ]:
app.create_actors_relationship_with_usecase("Energy saving controller", "uses", 
                                        "modules", "usecase_007_cat2")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "modules", "usecase_007_cat2")    
app.create_actors_relationship_with_usecase("data acquisition", "is", 
                                        "modules", "usecase_007_cat2")
app.create_actors_relationship_with_usecase("policy", "is", 
                                        "modules", "usecase_007_cat2")  
app.create_actors_relationship_with_usecase("Action implementation", "is", 
                                        "modules", "usecase_007_cat2")        
app.create_actors_relationship_with_usecase("Data acquisition", "uses", 
                                        "data center APIs", "usecase_007_cat2")   
app.create_actors_relationship_with_usecase("policy", "has", 
                                        "data center policy", "usecase_007_cat2")   
app.create_actors_relationship_with_usecase("Action implementation", "uses", 
                                        "data center APIs", "usecase_007_cat2")        
app.create_actors_relationship_with_usecase("controllers", "need", 
                                        "evaluation", "usecase_007_cat2")                
app.create_actors_relationship_with_usecase("evaluation", "has", 
                                        "metrics", "usecase_007_cat2")        
app.create_actors_relationship_with_usecase("metrics", "are", 
                                        "data center metrics", "usecase_007_cat2") 
app.create_actors_relationship_with_usecase("controllers", "need", 
                                        "optimization", "usecase_007_cat2")     
app.create_actors_relationship_with_usecase("optimization", "uses", 
                                        "evaluation", "usecase_007_cat2")                                                

In [14]:
app.find_all_usecase_actors_label("usecase_007_cat2")

Found actor-1: controllers
Found actor-2: policy
Found actor-3: optimization
Found actor-4: Energy saving controller
Found actor-5: modules
Found actor-6: data acquisition
Found actor-7: Action implementation
Found actor-8: Data acquisition
Found actor-9: data center APIs
Found actor-10: data center policy
Found actor-11: evaluation
Found actor-12: metrics
Found actor-13: data center metrics


### FG-AN-USECASE-008 cat1

In [ ]:
app.create_actors_relationship_with_usecase("CL automation framework", "discovers", "network parameters", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "discovers", "data", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "discovers", "KPI", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "deploys", "controllers", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("controllers", "provide", "optimization", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("KPI", "need", "optimization", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("use cases", "has", "KPI", "usecase_008_cat1")
          

In [ ]:
app.create_actors_relationship_with_usecase("Evolution controller", "discovers", "use cases", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "composes", "controllers", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("use cases", "has", "data", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("data", "has", "parameters", "usecase_008_cat1")


In [15]:
app.find_all_usecase_actors_label("usecase_008_cat1")

Found actor-1: controllers
Found actor-2: Evolution controller
Found actor-3: use cases
Found actor-4: data
Found actor-5: KPI
Found actor-6: CL automation framework
Found actor-7: optimization
Found actor-8: network parameters
Found actor-9: parameters


### FG-AN-USECASE-008 cat2

In [ ]:
app.create_actors_relationship_with_usecase("base station", "has", "UE location", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("mMiMO controller", "needs", "UE location", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("mMiMO controller", "has", "optimization", "usecase_008_cat2") 
app.create_actors_relationship_with_usecase("optimization", "has", "goals", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "system utilization", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "user experience", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "target RSRP/SINR distribution", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "optimal antenna parameters", "usecase_008_cat2")

In [16]:
app.find_all_usecase_actors_label("usecase_008_cat2")

Found actor-1: optimization
Found actor-2: base station
Found actor-3: UE location
Found actor-4: mMiMO controller
Found actor-5: goals
Found actor-6: system utilization
Found actor-7: user experience
Found actor-8: target RSRP/SINR distribution
Found actor-9: optimal antenna parameters


### FG-AN-USECASE-009 cat1

In [ ]:
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "controller characteristics", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "accepts", "intent", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "composes", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "evolves", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "evolves", "modules", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "implements", "evolution algorithms", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "modules", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "composes", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "deploys", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("controllers", "utilizes", "underlay specific APIs", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("edge controllers", "are", "controllers", "usecase_009_cat1")


In [17]:
app.find_all_usecase_actors_label("usecase_009_cat1")

Found actor-1: controllers
Found actor-2: Adaptation controller
Found actor-3: intent
Found actor-4: Evolution controller
Found actor-5: modules
Found actor-6: controller characteristics
Found actor-7: evolution algorithms
Found actor-8: underlay specific APIs
Found actor-9: edge controllers


### FG-AN-USECASE-009 cat2


In [ ]:
app.create_actors_relationship_with_usecase("Emergency management controller", "uses", "controllers", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("controllers", "detect", "emergencies", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("controllers", "optimize", "network resource allocation", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("Emergency management controller", "instantiates", "controllers", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("Emergency management controller", "input", "intent", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("Emergency management controller", "output", "intent", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("controllers", "need", "network domain", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("intent", "has", "ml pipeline", "usecase_009_cat2")


In [18]:
app.find_all_usecase_actors_label("usecase_009_cat2")

Found actor-1: controllers
Found actor-2: intent
Found actor-3: Emergency management controller
Found actor-4: emergencies
Found actor-5: network resource allocation
Found actor-6: network domain
Found actor-7: ml pipeline


### FG-AN-USECASE-010 cat1

In [ ]:
app.create_actors_relationship_with_usecase("AN orchestrator", "accepts", "intent", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Evolution controller", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Adaptation controller", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Experimentation controller", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("intent", "has", "vertical specific inputs", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Experimentation controller", "validates", "controllers", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", "controllers", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "produces", "reports", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("underlay", "are", "CI/CD pipeline", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "reference points", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("underlay", "has", "reference points", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "controllers", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay capabilities", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("underlay", "has", "underlay capabilities", "usecase_010_cat1")



In [19]:
app.find_all_usecase_actors_label("usecase_010_cat1")

Found actor-1: controllers
Found actor-2: reports
Found actor-3: Experimentation controller
Found actor-4: Adaptation controller
Found actor-5: underlay
Found actor-6: intent
Found actor-7: Evolution controller
Found actor-8: AN orchestrator
Found actor-9: vertical specific inputs
Found actor-10: CI/CD pipeline
Found actor-11: reference points
Found actor-12: underlay capabilities


### FG-AN-USECASE-010 cat2

In [ ]:
app.create_actors_relationship_with_usecase("Cloud service controller", "uses", "controllers", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "intent", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "simulation", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "validation", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("validation", "needs", "KPI", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("validation", "needs", "AN sandbox", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "output", "cloud service deployment", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("cloud service deployment", "needs", "optimization", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "output", "network service deployment", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("network service deployment", "needs", "optimization", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "monitors", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("monitors", "output", "reports", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("network service deployment", "needs", "technology migration", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("cloud service deployment", "needs", "technology migration", "usecase_010_cat2")




In [20]:
app.find_all_usecase_actors_label("usecase_010_cat2")

Found actor-1: cloud service deployment
Found actor-2: network service deployment
Found actor-3: monitors
Found actor-4: technology migration
Found actor-5: controllers
Found actor-6: reports
Found actor-7: AN sandbox
Found actor-8: intent
Found actor-9: KPI
Found actor-10: optimization
Found actor-11: Cloud service controller
Found actor-12: simulation
Found actor-13: validation


### FG-AN-USECASE-011 cat1

In [ ]:
app.create_actors_relationship_with_usecase("AN orchestrator", "input", "intent", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "domain orchestrator", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Evolution controller", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Adaptation controller", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("intent", "has", "controller design", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("intent", "has", "controller deployment", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("intent", "has", "controller management", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("controllers", "monitors", "inter-domain-connectivity", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("controllers", "configures", "inter-domain-connectivity", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("controllers", "manages", "inter-domain-connectivity", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("Evolution controller", "implements", "evolution algorithms", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("evolution algorithms", "implements", "evolution", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("controllers", "need", "evolution", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("underlay", "are", "inter-domain-connectivity", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "controllers", "usecase_011_cat1")

In [21]:
app.find_all_usecase_actors_label("usecase_011_cat1")

Found actor-1: domain orchestrator
Found actor-2: controller design
Found actor-3: controller deployment
Found actor-4: controller management
Found actor-5: inter-domain-connectivity
Found actor-6: evolution
Found actor-7: controllers
Found actor-8: Adaptation controller
Found actor-9: underlay
Found actor-10: intent
Found actor-11: Evolution controller
Found actor-12: evolution algorithms
Found actor-13: AN orchestrator


### FG-AN-USECASE-011 cat2

In [23]:
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "has", "alternatives", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("verticals", "provide", "service", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("Rural sites", "requirement", "service", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("verticals", "requirement", "inter-domain-connectivity", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("Rural sites", "requirement", "inter-domain-connectivity", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "is", "shared", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "requirement", "edge compute", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "requirement", "CL automation framework", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("edge compute", "requirement", "CL automation framework", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("edge compute", "has", "micro service", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("alternatives", "has", "heterogeneous", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "controllers", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "requirement", "management", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "onboarding", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("applications", "requirement", "onboarding", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "monitoring", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("applications", "requirement", "monitoring", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "report", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("human operator", "requirement", "report", "usecase_011_cat2")

Node already exists: inter-domain-connectivity in ['usecase_011_cat1', 'usecase_011_cat2']
ignoring ...
Node already exists: alternatives in ['usecase_011_cat2']
ignoring ...
has already exists with usecase_011_cat2 between inter-domain-connectivity and alternatives
Created node: verticals
Node already exists: service in ['usecase_011_cat2']
ignoring ...
Created relation: verticals - provide - service in usecase_011_cat2
Node already exists: Rural sites in ['usecase_011_cat2']
ignoring ...
Node already exists: service in ['usecase_011_cat2']
ignoring ...
requirement already exists with usecase_011_cat2 between Rural sites and service
Node already exists: verticals in ['usecase_011_cat2']
ignoring ...
Node already exists: inter-domain-connectivity in ['usecase_011_cat1', 'usecase_011_cat2']
ignoring ...
Created relation: verticals - requirement - inter-domain-connectivity in usecase_011_cat2
Node already exists: Rural sites in ['usecase_011_cat2']
ignoring ...
Node already exists: inter

In [24]:
app.find_all_usecase_actors_label("usecase_011_cat2")

Found actor-1: inter-domain-connectivity
Found actor-2: alternatives
Found actor-3: controllers
Found actor-4: CL automation framework
Found actor-5: underlay orchestrator
Found actor-6: service
Found actor-7: Rural sites
Found actor-8: verticals
Found actor-9: shared
Found actor-10: edge compute
Found actor-11: micro service
Found actor-12: heterogeneous
Found actor-13: management
Found actor-14: onboarding
Found actor-15: applications
Found actor-16: monitoring
Found actor-17: report
Found actor-18: human operator


### FG-AN-USECASE-012 cat1

In [10]:
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", "underlay capabilities", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay capabilities", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("underlay capabilities", "is", "underlay configuration", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("underlay capabilities", "is", "monitoring", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "capability", "recommendation", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("recommendation", "is", "underlay capabilities", "usecase_012_cat1")

Node already exists: Adaptation controller in ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1']
adding label usecase_012_cat1 in addition to ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1']
Node already exists: underlay capabilities in ['usecase_010_cat1']
adding label usecase_012_cat1 in addition to ['usecase_010_cat1']
Created relation: Adaptation controller - monitors - underlay capabilities in usecase_012_cat1
Node already exists: Adaptation controller in ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1']
ignoring ...
Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1']
ignoring ...
Created relation: Adaptation controller - discovers - underlay capabilities in usecase_012_cat1
Node already exists: underlay capabilities in ['usecase_010_cat1', 'useca

In [11]:
app.find_all_usecase_actors_label("usecase_012_cat1")

Found actor-1: Adaptation controller
Found actor-2: Evolution controller
Found actor-3: underlay capabilities
Found actor-4: underlay configuration
Found actor-5: monitoring
Found actor-6: recommendation


### FG-AN-USECASE-012 cat2

In [14]:
app.create_actors_relationship_with_usecase("Data analysis function", "capability", "data collector", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("Network function", "capability", "event report", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "UE IP address change", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "UP path change", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "PDU Session Establishment", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "PDU Session Release", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("Data analysis function", "capability", "Data analytics", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("Data analytics", "is", "Load information of UPF", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "data", "usecase_012_cat2")

Node already exists: Data analysis function in ['usecase_012_cat2']
ignoring ...
Node already exists: data collector in ['usecase_012_cat2']
ignoring ...
capability already exists with usecase_012_cat2 between Data analysis function and data collector
Node already exists: Network function in ['usecase_012_cat2']
ignoring ...
Node already exists: event report in ['usecase_012_cat2']
ignoring ...
capability already exists with usecase_012_cat2 between Network function and event report
Node already exists: event report in ['usecase_012_cat2']
ignoring ...
Node already exists: UE IP address change in ['usecase_012_cat2']
ignoring ...
is already exists with usecase_012_cat2 between event report and UE IP address change
Node already exists: event report in ['usecase_012_cat2']
ignoring ...
Node already exists: UP path change in ['usecase_012_cat2']
ignoring ...
is already exists with usecase_012_cat2 between event report and UP path change
Node already exists: event report in ['usecase_012_c

In [15]:
app.find_all_usecase_actors_label("usecase_012_cat2")

Found actor-1: data
Found actor-2: Data analytics
Found actor-3: Data analysis function
Found actor-4: data collector
Found actor-5: Network function
Found actor-6: event report
Found actor-7: UE IP address change
Found actor-8: UP path change
Found actor-9: PDU Session Establishment
Found actor-10: PDU Session Release
Found actor-11: Load information of UPF


### FG-AN-USECASE-013 cat1

In [16]:

app.create_actors_relationship_with_usecase("Network function", "has", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay requirements", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "capability", "recommendation", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("recommendation", "is", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("recommendation", "is", "underlay requirements", "usecase_013_cat1")

Node already exists: Network function in ['usecase_012_cat2']
adding label usecase_013_cat1 in addition to ['usecase_012_cat2']
Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1']
adding label usecase_013_cat1 in addition to ['usecase_010_cat1', 'usecase_012_cat1']
Created relation: Network function - has - underlay capabilities in usecase_013_cat1
Node already exists: Adaptation controller in ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1']
adding label usecase_013_cat1 in addition to ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1']
Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1', 'usecase_013_cat1']
ignoring ...
Created relation: Adaptation controller - monitors - underlay capabilities in usecase_013_cat1
Node already exists: Adaptation controller in 

In [17]:
app.find_all_usecase_actors_label("usecase_013_cat1")

Found actor-1: Adaptation controller
Found actor-2: Evolution controller
Found actor-3: underlay capabilities
Found actor-4: recommendation
Found actor-5: Network function
Found actor-6: underlay requirements


### FG-AN-USECASE-013 cat2

In [18]:
app.create_actors_relationship_with_usecase("controllers", "requirement", "provisioning", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("provisioning", "is", "multi-domain", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("provisioning", "requirement", "underlay capabilities", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "requirement", "plug and play", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "is", "Network function", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "is", "network slice", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "is", "network service", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "capability", "data collection", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("controllers", "requirement", "bootstrap", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network infrastructure", "requirement", "bootstrap", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network service", "requirement", "bootstrap", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("applications", "requirement", "bootstrap", "usecase_013_cat2")

Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2']
adding label usecase_013_cat2 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2']
Created node: provisioning
Created relation: controllers - requirement - provisioning in usecase_013_cat2
Node already exists: provisioning in ['usecase_013_cat2']
ignoring ...
Created node: multi-domain
Created relation: provisioning - is - multi-domain in usecase_013_cat2
Node already exists: provisioning in ['usecase_013_cat2']
ignoring ...
Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1', 'usecase_013_cat1']
addi

In [19]:
app.find_all_usecase_actors_label("usecase_013_cat2")

Found actor-1: controllers
Found actor-2: underlay capabilities
Found actor-3: applications
Found actor-4: Network function
Found actor-5: provisioning
Found actor-6: multi-domain
Found actor-7: network instance
Found actor-8: plug and play
Found actor-9: network slice
Found actor-10: network service
Found actor-11: data collection
Found actor-12: bootstrap
Found actor-13: network infrastructure


### FG-AN-USECASE-014

In [21]:
app.create_actors_relationship_with_usecase("underlay", "are", "hybrid", "usecase_014")
app.create_actors_relationship_with_usecase("hybrid", "capability", "simulators", "usecase_014")
app.create_actors_relationship_with_usecase("hybrid", "capability", "test network", "usecase_014")
app.create_actors_relationship_with_usecase("hybrid", "capability", "real network", "usecase_014")
app.create_actors_relationship_with_usecase("multi-domain", "capability", "hybrid", "usecase_014")

Node already exists: underlay in ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1']
adding label usecase_014 in addition to ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1']
Created node: hybrid
Created relation: underlay - are - hybrid in usecase_014
Node already exists: hybrid in ['usecase_014']
ignoring ...
Node already exists: simulators in ['usecase_002']
adding label usecase_014 in addition to ['usecase_002']
Created relation: hybrid - capability - simulators in usecase_014
Node already exists: hybrid in ['usecase_014']
ignoring ...
Created node: test network
Created relation: hybrid - capability - test network in usecase_014
Node already exists: hybrid in ['usecase_014']
ignoring ...
Created node: real network
Created relation: hybrid - capability - real network in usecase_014
Node already exists: multi-domain in ['usecase_013_cat2']
adding label usecase_014 in addition to ['usecase_013_cat2']
Node already exists: hybrid in ['usecase_014']
ignoring ...
Created relatio

In [22]:
app.find_all_usecase_actors_label("usecase_014")

Found actor-1: simulators
Found actor-2: underlay
Found actor-3: multi-domain
Found actor-4: hybrid
Found actor-5: test network
Found actor-6: real network


### FG-AN-USECASE-015

In [ ]:
app.create_actors_relationship_with_usecase("data", "is", "logs", "usecase_015")
app.create_actors_relationship_with_usecase("Network instance", "capability", "logs", "usecase_015")
app.create_actors_relationship_with_usecase("model repository", "capability", "ML model", "usecase_015")
app.create_actors_relationship_with_usecase("model repository", "is", "3rd party", "usecase_015")


In [ ]:
app.find_all_usecase_actors_label("usecase_015")

## Final Step: close the database connection

In [ ]:
app.close()